In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

torch.manual_seed(0)
import random
random.seed(0)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
path_train      = 'F://TFG//datasets//data_train//'
path_graphs     = 'F://TFG//graphs//'

In [5]:
data = pd.read_csv(path_train+'training_features_DF.csv',sep=';',index_col='wyId')

# X_train = pd.read_csv(path_train+'X_train.csv',sep=';',index_col='wyId')
# y_train = pd.read_csv(path_train+'y_train.csv',sep=';',index_col='wyId')
# X_test = pd.read_csv(path_train+'X_test.csv',sep=';',index_col='wyId')
# y_test = pd.read_csv(path_train+'y_test.csv',sep=';',index_col='wyId')

In [6]:
scaler  = preprocessing.StandardScaler()
X_std   = scaler.fit_transform(X_train)
X_std.mean(), X_std.std()

NameError: name 'X_train' is not defined

In [26]:
class FootballMatchesDataset(Dataset):
    def __init__(self,file):
        df              = pd.read_csv(path_train+'X_'+file+'.csv',sep=';')
        lab_df          = pd.read_csv(path_train+'y_'+file+'.csv',sep=';')
        self.data       = torch.tensor(df.values[:,1:]) 
        self.labels     = F.one_hot(torch.tensor(lab_df.values[:,1]), num_classes=3)
        self.matches    = torch.tensor(lab_df.values[:,0])

    def __len__(self):
        return len(self.data)

    def shape(self):
        return self.data.shape

    def __getitem__(self,idx):
        sample  = self.data[idx]
        label   = self.labels[idx]
        match   = self.matches[idx]
        return sample, label, match



In [27]:
train_data  = FootballMatchesDataset(file = 'train')
test_data   = FootballMatchesDataset(file = 'test')

In [29]:
dataloader = DataLoader(train_data, batch_size=5, shuffle=True, num_workers=0)
train_feat, train_lab, _ = next(iter(dataloader))
train_lab

tensor([[0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 1, 0]])

In [30]:
train_data.data = scaler.fit_transform(train_data.data)

### Neural Network Implementation

Define the class:

#### I) Artificial Neural Network Approach to Football Score Prediction

Multilayer Perceptron with 1 hidden layer with BacpPropagation.
6 units input -> 5 hidden units -> 2 output units w/ sigmoid

Data Normalized [0,1]

In [31]:
train_data  = FootballMatchesDataset(file = 'train')
test_data   = FootballMatchesDataset(file = 'test')

In [32]:
normalizer = preprocessing.Normalizer()
train_data.data = normalizer.fit_transform(train_data.data)

In [33]:
print(train_data.data.mean(), train_data.data.std())
print(train_data.data.max(),  train_data.data.min())

0.06812330356942047 0.2020241593605429
0.9647154754393831 0.0


In [34]:
dataloader_train    = DataLoader(train_data, batch_size=5, shuffle=True)
dataloader_test     = DataLoader(test_data,  batch_size=5, shuffle=True)

train_feat, train_lab, _ = next(iter(dataloader_train))
train_feat

tensor([[4.2946e-02, 2.8027e-02, 1.0669e-02, 1.3179e-02, 7.3833e-05, 2.0919e-04,
         6.2758e-04, 1.2552e-03, 3.1379e-04, 1.7931e-04, 7.2799e-01, 6.8343e-01,
         5.0964e-04, 5.3710e-04, 1.8827e-03, 4.3931e-03, 2.2593e-04, 1.5062e-04,
         6.5268e-04, 3.0124e-04, 5.4390e-04, 8.9044e-04],
        [0.0000e+00, 0.0000e+00, 2.2056e-02, 9.6496e-03, 1.9385e-04, 3.4463e-04,
         1.3785e-03, 1.3785e-03, 1.7231e-04, 3.4463e-04, 8.8983e-01, 4.5560e-01,
         5.8728e-04, 5.1095e-04, 6.2033e-03, 2.7570e-03, 1.4061e-03, 6.0654e-04,
         6.3411e-04, 1.5991e-03, 8.7470e-04, 6.9711e-04],
        [4.3620e-02, 2.9954e-02, 1.6196e-02, 2.4969e-02, 2.8119e-04, 2.3711e-04,
         2.6994e-03, 1.3497e-03, 3.3743e-04, 6.7485e-04, 7.3356e-01, 6.7688e-01,
         5.4696e-04, 5.3625e-04, 2.6994e-03, 4.0491e-03, 1.1068e-03, 1.3497e-03,
         2.4295e-04, 8.0982e-04, 4.3584e-04, 5.7291e-04],
        [3.5134e-02, 3.1048e-02, 1.6555e-02, 1.7126e-02, 1.5748e-04, 1.7126e-04,
         2.2835e

In [74]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_feature, ouput_classes):
        super().__init__()
        
        self.h1 = nn.Linear(in_features=input_feature,out_features=5)
        self.bn = nn.BatchNorm1d(5)
        self.out = nn.Linear(5,ouput_classes)

    def forward(self,x):
        x = F.relu(self.h1(x))
        x = self.bn(x)
        return F.softmax(self.out(x))        

In [75]:
model = NeuralNetwork(22,3)

In [76]:
# Print out the architecture and number of parameters.
print(model)
print(f"The model has {sum([x.nelement() for x in model.parameters()]):,} parameters.")

NeuralNetwork(
  (h1): Linear(in_features=22, out_features=5, bias=True)
  (bn): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=5, out_features=3, bias=True)
)
The model has 143 parameters.


##### Loss Function: Cross-entropy Loss

we can provide `weights`, as prior probability of each class $C$.

In [77]:
train_data.labels   # in 1-hot encoding

tensor([[0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        ...,
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0]])

In [78]:
weights_class = np.mean(train_data.labels.numpy(),axis=0)

criterion = nn.CrossEntropyLoss()

##### Optimizer

In [79]:
learning_rate = 1e-1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# optimizar con momentum (nesterov), weight decay

##### Accuracy

In [80]:
def accuracy(pred,y):
    pred_class = torch.argmax(pred,dim=1)
    torch.mean((pred_class == y)*1.0)

##### Train Loop

In [81]:
def train_model(model, criterion, optimizer, dataloader_train, dataloader_test, epochs):
    for ep in range(epochs):
        # Training.
        model.train()
        for it, batch in enumerate(dataloader_train):
            # 5.1 Load a batch, break it down in images and targets.
            x, y, _ = batch
            # batch to device ????

            # 5.2 Run forward pass.
            logits = model(x)
            
            # 5.3 Compute loss (using 'criterion').
            loss = criterion(logits, y)
            
            # 5.4 Run backward pass.
            loss.backward()
            # `torch.autograd.backward(loss)` also works
            
            # 5.5 Update the weights using optimizer.
            optimizer.step()
            
            # 5.6 Zero-out the accumulated gradients.
            optimizer.zero_grad()
            # `model.zero_grad()` also works

            print('\rEp {}/{}, it {}/{}: loss train: {:.2f}, accuracy train: {:.2f}'.
                  format(ep + 1, epochs, it + 1, len(dataloader_train), loss,
                         accuracy(logits, y)), end='')

        # Validation.
        model.eval()
        with torch.no_grad():
            acc_run = 0
            for it, batch in enumerate(dataloader_test):
                # Get batch of data.
                x, y = batch
                curr_bs = x.shape[0]
                acc_run += accuracy(model(x), y) * curr_bs
            acc = acc_run / len(dataloader_test.dataset)

            print(', accuracy test: {:.2f}'.format(acc))

In [82]:
# Train the model
epochs = 5
learning_rate = 1e-1
optimizer_lenet = torch.optim.SGD(model.parameters(), lr=learning_rate)
train_model(model, criterion, optimizer, dataloader_test, dataloader_test, epochs)

RuntimeError: expected scalar type Float but found Double